In [4]:
model_path = "../models/[500epoch]custom_t5_model.keras"
tokenizer_path = '../models/tokenizer.pkl'
max_news_len = 2025
max_summary_len = 100

In [5]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

In [ ]:
model = load_model('../models/[500epoch]custom_t5_model.keras')
with open(tokenizer_path, 'rb') as file:
    tokenizer = pickle.load(file)

In [ ]:
# def summarize_news(news_article):
#     news_sequence = tokenizer.texts_to_sequences([news_article])
#     news_padded = pad_sequences(news_sequence, maxlen=max_news_len, padding='post')
#     summary_sequence = model.predict([news_padded, decoder_input_data[:1]])
#     summary = tokenizer.sequences_to_texts(summary_sequence.argmax(axis=-1))
#     return summary[0]

In [ ]:
def summarize_news(news_article):
    # Tokenize and pad the news article
    news_sequence = tokenizer.texts_to_sequences([news_article])
    news_padded = pad_sequences(news_sequence, maxlen=max_news_len, padding='post')
    
    # Initialize the decoder input with the start token (assuming 0 is the start token)
    start_token = np.zeros((1, 1))
    
    # Placeholder for the generated summary
    generated_summary = []

    # Iteratively generate tokens
    for _ in range(max_summary_len - 1):
        # Predict the next token
        predictions = model.predict([news_padded, start_token])
        next_token = np.argmax(predictions[0, -1, :])
        
        # Append the predicted token to the generated summary
        generated_summary.append(next_token)
        
        # Update the decoder input with the new token
        start_token = np.hstack([start_token, np.array([[next_token]])])
        
        # Break if the end token is generated (assuming 0 is the end token)
        if next_token == 0:
            break
    
    # Convert the generated token IDs to text
    summary = tokenizer.sequences_to_texts([generated_summary])
    
    return summary[0]

# Example usage
news_article = """Addressing public complaints, Prime Minister KP Sharma Oli has said that the government's top priority is how to conduct agile, sound, competent and transparent administration.

In the 53rd meeting of the National Development Problem Solution Committee held at Singhdarbar on Friday, Prime Minister Oli said that since good governance is more necessary for development, he has put governance reform at the top of the agenda to be decided by the committee.

He said that while protecting the forest environment, development projects should be facilitated in this area and forest-related agencies should not work in such a way as to hinder development. He referred to the fact that the Ministry of Forestry had sent more staff than necessary to the conference on climate in Baku, Azerbaijan, and drew the attention of ministers and secretaries that such wrong trends should not happen.

'We will have to act with speed and make bold decisions . Production and productivity should be increased by modernizing agriculture and encouraging the private sector. Acts of taunting and abusing entrepreneurs and getting applause with similar tendencies should be strictly discouraged,' Prime Minister Oli said, 'We should work by creating an environment for investment in business and industry . As a very big conspiracy, social chaos has been caused to hinder development.

Prime Minister Oli instructed the Ministry of Education to come up with alternative plans including online classes to improve the results of Secondary Education Examination Class 10 (SEE).

Government reforms presented at the meeting, use of forest area in national priority projects, project bank, district rates for public procurement purposes, operation of children's correctional homes and free street children campaign, government buildings at federal, state and local levels, supplementary grants, sixteen projects Implementation plan, multi-year contract management of projects, electricity tariff collection regulations, compulsory and free education, internship in public service, disaster resilient infrastructure construction, post-disaster reconstruction and rehabilitation. Decisions have been made in the agendas with solution options. In the

meeting, the Vice Chairman of the National Planning Commission, Shivraj Adhikari, discussed the state and trend of macroeconomic indicators, the state of Nepal government's public finance mobilization, the state of economic and social progress after federalism up to the current financial year, the state of the overall economic state of the provinces, the state of progress of the current financial year, and the previous three development problems. He made a presentation on the implementation status of the decision of the meeting of the solution committee.

Deputy Prime Minister and Finance Minister Bishnu Prasad Paudel, Ministers, Chief Ministers of State Governments, Chief Advisor to the Prime Minister, Governor of Nepal Rastra Bank, Chief Secretary of the Government of Nepal, Officials of the National Natural Resources and Finance Commission, Auditor General, Secretary of the Union and State Governments and others participated in the"""

generated_summary = summarize_news(news_article)
print("Generated Summary:", generated_summary)
